# Post-processing of case studies

## Usage notes

All figures will be written to the directory `img` as pdf and png files.

I run this notebook on Ubuntu, using this version of matplotlib:
```
pip freeze | grep matplotlib
matplotlib==2.1.0
```
Some older versions return an error because they do not support some plot configurations.
I set in the virtual machine the RAM to 12GB as 8GB is not sufficient to parse two annual result files.

## Import required libraries

In [6]:
import os
import cases

import matplotlib.pyplot as plt

from buildingspy.io.outputfile import Reader
from buildingspy.io.postprocess import Plotter

## Configure plots

In [7]:
plt.rcParams['axes.facecolor']='whitesmoke'
plt.rcParams['font.size'] = 6
plt.rcParams['text.usetex'] = False
plt.rcParams['legend.facecolor'] = 'white'
plt.rcParams['legend.framealpha'] = 0.75
plt.rcParams['legend.edgecolor'] = 'none'
plt.rcParams['savefig.dpi'] = 300

def save_plot(figure, file_name):
    """ Save the figure to a pdf and png file in the directory `img`
    """
    import os
    out_dir = "img"
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    figure.savefig(os.path.join(out_dir, '{}.pdf'.format(file_name)))
    figure.savefig(os.path.join(out_dir, '{}.png'.format(file_name)))  

def configure_axes(axes):
    """ Configure the axis style
    """
    axes.spines['right'].set_visible(False)
    axes.spines['top'].set_visible(False)
    axes.spines['left'].set_visible(False)
    axes.spines['bottom'].set_visible(False)
    axes.grid(color='lightgrey', linewidth=0.25)
    return

def get_results(case_name):
    """ Get the results for the case with name `case_name`
    """
    # Make sure simulation was successful
    dslog_name = os.path.join(case_name, "dslog.txt")
    with open(dslog_name) as dslog:
        if not "Integration terminated successfully" in dslog.read():
            raise Exception("Simulation failed. Check {}".format(dslog_name))
    file_name = cases.get_result_file_name(case_name)
    return Reader(file_name, "dymola")

# ---------------------------------------------------------------------------
# helper functions and scripts

def set_cases_and_initiate_plot():
    cases = ['Base case', 'Guideline 36']
    seasons = ['Winter', 'Spring', 'Summer']
    num_cases = len(cases)
    num_seasons = len(seasons)
    
    fig, ax = plt.subplots(nrows=num_cases*num_seasons, ncols=1, figsize = (6.5,8.))
    
    return cases, seasons, num_cases, num_seasons, fig, ax

def set_up_labels(i, ax, cases, seasons, num_cases, num_seasons, x_axis_label, y_axis_label):
        # Hide xtick labels and ticks on the upper case subplot (each basecase)
        if i % 2 == 0:
            hide_tick_labels(ax)

        # Print x axis title only below the lowest subplot
        if i  == num_cases*num_seasons - 1:
            ax.set_xlabel(x_axis_label)
        ax.set_ylabel(y_axis_label)
        #ax.xaxis.set_ticks(np.arange(min(t)+0, 365, 1))
     
        # Annotate case
        if i % 2 == 0:
            title_str = seasons[i % 3] + ' (upper: ' + cases[i % 2] + ', lower: ' + cases[(i-1) % 2] + ')'
            ax.set_title(title_str, # mg assign appropriate season/case
                         verticalalignment = 'top',
                         horizontalalignment = 'center', 
                         fontsize = 6, color = 'k')
        
        # Print legend only at the lower plot (g36 case)
        if i % 2 == 1:
            ax.legend(loc='center right', ncol=1)
        configure_axes(ax)
        
        #plt.tight_layout(h_pad=0)
        plt.tight_layout()
        #plt.subplots_adjust(hspace = .2)
        
def tem_conv_CtoF(T_in_degC):
    '''Converts temperature provided in degC to degF
    '''
    T_in_degF = (T_in_degC)*9./5. + 32.
    
    return T_in_degF
        
def add_secondary_yaxis_for_degF(ax, time, temp_in_K):
        # Add a secondary axis with temperatures represented in F
        ax_F = ax.twinx()
        # Get limits to match with the left axis
        ax_F.set_ylim([tem_conv_CtoF(ax.get_ylim()[0]),tem_conv_CtoF(ax.get_ylim()[1])])
        # plot a "scaler" variable and make it invisible
        ax_F.plot(time, tem_conv_CtoF(temp_in_K-273.15), linewidth=0.0)
        ax_F.set_ylabel('temperature [$^\circ$F]')
        configure_axes(ax_F)
        ax.grid(False)
        ax.xaxis.grid()
        
def hide_tick_labels(ax):
    '''Removes labels and ticks. Kwargs: bottom controls the ticks, labelbottom the tick labels
    '''
    ax.tick_params(axis = 'x',labelbottom='off',bottom='off')

## Read data of short-term simulations

In [8]:
# Cases and seasones combined
winter_base = 0
winter_g36 = 1
spring_base = 2
spring_g36 = 3
summer_base = 4
summer_g36 = 5

r_all_cases_and_seasons = list()

suffix = ''
#suffix = '_diverse_loads'

r_all_cases_and_seasons.insert(winter_base, Reader(cases.get_result_file_name('winter_base'+suffix), "dymola"))
r_all_cases_and_seasons.insert(winter_g36, Reader(cases.get_result_file_name('winter_g36'+suffix), "dymola"))
r_all_cases_and_seasons.insert(spring_base, Reader(cases.get_result_file_name('spring_base'+suffix), "dymola"))
r_all_cases_and_seasons.insert(spring_g36, Reader(cases.get_result_file_name('spring_g36'+suffix), "dymola"))
r_all_cases_and_seasons.insert(summer_base, Reader(cases.get_result_file_name('summer_base'+suffix), "dymola"))
r_all_cases_and_seasons.insert(summer_g36, Reader(cases.get_result_file_name('summer_g36'+suffix), "dymola"))

## Plot results

### Room temperatures

In [5]:
def plot_room_temp(readers):
    ''' Main method that plots the results
    '''
    from buildingspy.io.outputfile import Reader
    import matplotlib.pyplot as plt
    import matplotlib

    # Optionally, change fonts to use LaTeX fonts
    #from matplotlib import rc
    #rc('text', usetex=True)
    #rc('font', family='serif')
    
    font = {'family' : 'serif',
            'weight' : 'normal',
            'size'   : 6}
    matplotlib.rc('font', **font)
    
    plt.clf()
    
    time_scale=86400.
    
    cases, seasons, num_cases, num_seasons, fig, ax = set_cases_and_initiate_plot()
    
    # Read results
    for i in range(num_cases*num_seasons):
        (t, TOut) = readers[i].values("weaBus.TDryBul")
        t = t/time_scale

        TRoo = list()
        reader=readers[i]
        TRoo.insert(0, reader.values("flo.nor.air.vol.T")[1])
        TRoo.insert(1, reader.values("flo.wes.air.vol.T")[1])
        TRoo.insert(2, reader.values("flo.sou.air.vol.T")[1])
        TRoo.insert(3, reader.values("flo.eas.air.vol.T")[1])
        TRoo.insert(4, reader.values("flo.cor.air.vol.T")[1])

        TSetHea = reader.values("conVAVCor.TRooHeaSet")[1]
        TSetCoo = reader.values("conVAVCor.TRooCooSet")[1]

        # Generate figure and plot data
        ax[i] = plt.subplot(num_cases*num_seasons, 1, i+1)

        ax[i].plot(t, TOut-273.15,    'k', label='$T_{out}$', linewidth=0.5)
        ax[i].plot(t, TRoo[0]-273.15, 'b',  label='$T_{nor}$', linewidth=0.5)
        ax[i].plot(t, TRoo[1]-273.15, 'g',  label='$T_{wes}$', linewidth=0.5)
        ax[i].plot(t, TRoo[2]-273.15, 'r',  label='$T_{sou}$', linewidth=0.5)
        ax[i].plot(t, TRoo[3]-273.15, 'c',  label='$T_{eas}$', linewidth=0.5)
        ax[i].plot(t, TRoo[4]-273.15, 'g',  label='$T_{cor}$', linewidth=0.5)

        ax[i].patch.set_facecolor('mistyrose')    
        ax[i].fill_between(t, TSetHea-273.15, y2=TSetCoo-273.15, color='white')
        
        # customize days to display
        ax[i].set_xlim([min(t)+5, min(t)+12])
        
        set_up_labels(i, ax[i], 
                      cases, seasons, num_cases, num_seasons, 
                      'time [days]', 'temperature [$^\circ$C]')
        
        add_secondary_yaxis_for_degF(ax[i], t, TOut)

    return plt

# Create the plot for all seasons and cases
fig = plot_room_temp(r_all_cases_and_seasons)
save_plot(fig, "TRoom_all")

### VAV control

In [6]:
def plot_vav(readers):
    ''' Main method that plots the results
    '''
    # ------------------------------------------------------
    from buildingspy.io.outputfile import Reader
    import matplotlib.pyplot as plt

    room_styles = ['k', 'b', 'g', 'r', 'c', 'g']
    room_names = ['nor', 'wes', 'sou', 'eas', 'cor']  
    plt.clf()   
    time_scale=86400.
    # ------------------------------------------------------
    
    cases, seasons, num_cases, num_seasons, fig, ax = set_cases_and_initiate_plot()
    
    # Read results
    for i in range(num_cases*num_seasons):
        (t, TOut) = readers[i].values("weaBus.TDryBul")
        t = t/time_scale
        
        yHea = list()
        yVAV = list()
        
        reader=readers[i]
        
        for kRoo in range(len(room_names)):
            yHea.insert(kRoo, reader.values(\
                "{}.yVal".format(room_names[kRoo]))[1])
            yVAV.insert(kRoo, reader.values(\
                "{}.yVAV".format(room_names[kRoo]))[1])

        # Generate figure and plot data
        ax[i] = plt.subplot(num_cases*num_seasons, 1, i+1)

        for iRoo in [0, 2]:
            ax[i].plot(t, yHea[iRoo], room_styles[iRoo], \
                   label='$y_{{hea,{name}}}$'.format(name=room_names[iRoo]), linewidth=0.25)
            ax[i].plot(t, yVAV[iRoo], room_styles[iRoo+1], \
                    label='$y_{{vav,{name}}}$'.format(name=room_names[iRoo]), linewidth=0.25)
            
        # customize days to display
        ax[i].set_xlim([min(t)+5, min(t)+12])

        set_up_labels(i, ax[i], 
                      cases, seasons, num_cases, num_seasons, 
                      'time [days]', 'control signal [$1$]')

    return plt

# Create the plot for all seasons and cases
fig = plot_vav(r_all_cases_and_seasons)
save_plot(fig, "vav_all")

### AHU temperatures

In [8]:
def plot_ahu_temp(readers):
    ''' Main method that plots the results
    '''
    # ------------------------------------------------------
    from buildingspy.io.outputfile import Reader
    import matplotlib.pyplot as plt

    room_styles = ['k', 'b', 'g', 'r', 'c', 'g']
    room_names = ['nor', 'wes', 'sou', 'eas', 'cor']
    plt.clf()   
    time_scale=86400.
    # ------------------------------------------------------

    cases, seasons, num_cases, num_seasons, fig, ax = set_cases_and_initiate_plot()
    
    # Read results
    for i in range(num_cases*num_seasons):
        (t, TOut) = readers[i].values("weaBus.TDryBul")
        t = t/time_scale
        
        reader=readers[i]
        
        TSup = reader.values("TSup.T")[1]
        TMix = reader.values("TMix.T")[1]
        TRet = reader.values("TRet.T")[1]
  
        # Generate figure and plot data
        ax[i] = plt.subplot(num_cases*num_seasons, 1, i+1)

        ax[i].plot(t, TOut-273.15,    'gray', label='$T_{out}$', linewidth=0.25)
        ax[i].plot(t, TSup-273.15,    'r', label='$T_{sup}$', linewidth=0.25)
        ax[i].plot(t, TMix-273.15,    'g', label='$T_{mix}$', linewidth=0.25)
        ax[i].plot(t, TRet-273.15,    'b', label='$T_{ret}$', linewidth=0.25)

        ax[i].set_xlim([min(t)+5, min(t)+12])
 
        set_up_labels(i, ax[i], 
                      cases, seasons, num_cases, num_seasons, 
                      'time [days]', 'temperature [$^\circ$C]')
        
        add_secondary_yaxis_for_degF(ax[i], t, TOut)

    return plt

# Create the plot for all seasons and cases
fig = plot_ahu_temp(r_all_cases_and_seasons)
save_plot(fig, "TAHU_all")

Mixed air temperature with and without freeze protection

In [11]:
def plot_ahu_temp(readers):
    ''' Main method that plots the results
    '''
    # ------------------------------------------------------
    from buildingspy.io.outputfile import Reader
    import matplotlib.pyplot as plt

    plt.clf()   
    time_scale=86400.
    # ------------------------------------------------------

    # Read results
    
    for i in [0, 1]:
        (t, TOut) = readers[i].values("weaBus.TDryBul")
        t = t/time_scale
        
        reader=readers[i]
        
        if i == 0:
            (tWith, TMixWith) = reader.values("TMix.T")
            yEcoOutWith = reader.values("eco.yOut")[1]
            
            tWith = tWith/time_scale
        else:
            TMixNoFr = reader.values("TMix.T")[1]
            yEcoOutNoFr = reader.values("eco.yOut")[1]
            TSupNoFr = reader.values("TSup.T")[1]

  
    # Plot figure

    ax = plt.subplot(2, 1, 1)
    ax.plot(t, TOut-273.15,        'gray', label='$T_{out}$', linewidth=0.25)
    ax.plot(t, TSupNoFr-273.15,    'b', label='$T_{sup}$', linewidth=0.25)
    ax.plot(tWith, TMixWith-273.15,    'g', label='$T_{mix,with}$', linewidth=0.5)
    ax.plot(t, TMixNoFr-273.15,    'r', label='$T_{mix}$',  linewidth=0.5)

    #ax.set_xlabel('time [days]')
    ax.set_ylabel('temperature [$^\circ$C]')
    #ax.xaxis.set_ticks(np.arange(min(t)+0, 365, 1))

    ax.set_xlim([min(t)+5, min(t)+12])
 
    ax.legend(loc='center right', ncol=1)

    configure_axes(ax)

    ax = plt.subplot(2, 1, 2)
    ax.plot(tWith, yEcoOutWith,    'g', label='$y_{eco,out,with}$', linewidth=0.25)
    ax.plot(t, yEcoOutNoFr,    'r', label='$y_{eco,out,no}$',  linewidth=0.25)
    ax.set_xlabel('time [days]')
    ax.set_ylabel('$y_{eco,out}$ [$1$]')
    #ax.xaxis.set_ticks(np.arange(min(t)+0, 365, 1))

    ax.set_xlim([min(t)+5, min(t)+12])
 
    ax.legend(loc='center right', ncol=1)

    configure_axes(ax)
    
    return plt

# Create the plots
fig = plot_ahu_temp(r_all_cases_and_seasons[winter_g36], r_g36_noFreezeControl)
save_plot(fig, "TMixFre")

NameError: name 'r_g36_noFreezeControl' is not defined

### Flow rates and fan speed

In [13]:
def plot_flow_signals(readers):
    ''' Main method that plots the results
    '''
    # ------------------------------------------------------
    from buildingspy.io.outputfile import Reader
    import matplotlib.pyplot as plt

    room_styles = ['k', 'b', 'g', 'r', 'c', 'g']
    room_names = ['nor', 'wes', 'sou', 'eas', 'cor']  
    plt.clf()   
    time_scale=86400.
    # ------------------------------------------------------

    cases, seasons, num_cases, num_seasons, fig, ax = set_cases_and_initiate_plot()
    
    # Read results
    for i in range(num_cases*num_seasons):
        (t, TOut) = readers[i].values("weaBus.TDryBul")
        t = t/time_scale
        
        reader=readers[i]
        
        yFanSup = reader.values("fanSup.y")[1]
        #yFanRet = reader.values("fanRet.y")[1]
        #yEcoOut = reader.values("eco.yOut")[1]
        #yEcoRet = reader.values("eco.yRet")[1]
  
        # Generate figure and plot data
        ax[i] = plt.subplot(num_cases*num_seasons, 1, i+1)

        ax[i].plot(t, yFanSup, 'r', label='$y_{fan,sup}$', linewidth=0.25)
        #ax[i].plot(t, yFanRet, 'g', label='$y_{fan,ret}$', linewidth=0.25)
        #ax[i].plot(t, yEcoOut, 'g', label='$y_{eco,out}$', linewidth=0.25)
        #ax[i].plot(t, yEcoRet, 'b', label='$y_{eco,ret}$', linewidth=0.25)
        
        ax[i].set_xlim([min(t)+5, min(t)+12])
        
        set_up_labels(i, ax[i], 
                      cases, seasons, num_cases, num_seasons, 
                      'time [days]', 'control signal [$1$]')

    return plt

# Create the plots
fig = plot_flow_signals(r_all_cases_and_seasons)
save_plot(fig, "flow_signals_all")

### Normalized flow rates

In [14]:
def plot_normalized_flow_rates(readers):
    ''' Main method that plots the results
    '''
    # ------------------------------------------------------
    from buildingspy.io.outputfile import Reader
    import matplotlib.pyplot as plt

    room_styles = ['k', 'b', 'g', 'r', 'c', 'g']
    room_names = ['nor', 'wes', 'sou', 'eas', 'cor']  
    plt.clf()   
    time_scale=86400.
    # ------------------------------------------------------

    cases, seasons, num_cases, num_seasons, fig, ax = set_cases_and_initiate_plot()
    
    # Read results
    for i in range(num_cases*num_seasons):
        (t, TOut) = readers[i].values("weaBus.TDryBul")
        t = t/time_scale
        
        reader=readers[i]
        
        mFanSup = reader.values("fanSup.m_flow")[1]
#        mFanRet = reader.values("fanRet.m_flow")[1]/ \
#            reader.values("fanRet.m_flow_nominal")[1][1]
        mEco = reader.values("eco.port_Out.m_flow")[1]
        
        # Convert to air change per hours
        VBui = reader.values("ATot")[1][1] * reader.values("flo.hRoo")[1][1]
        print("Total floor area {}".format(reader.values("ATot")[1][1]))
        print("Room height {}".format(reader.values("flo.hRoo")[1][1]))
        print("Volume      {}".format(VBui))
        conFacACH = 3600./VBui/1.2
        
        fanSupACH = mFanSup * conFacACH
        ecoACH    = mEco    * conFacACH
        
        ax[i] = plt.subplot(num_cases*num_seasons, 1, i+1)

        ax[i].plot(t, fanSupACH,    'r', label='$\dot m_{fan,sup}/V_{bui}$', linewidth=0.25)
#        ax[i].plot(t, mFanRet,    'g', label='$\dot m_{fan,ret}/\dot m_{0}$', linewidth=0.25)
        ax[i].plot(t, ecoACH,       'g', label='$\dot m_{eco,out}/V_{bui}$', linewidth=0.25)

        
        # customize days to display
        ax[i].set_xlim([min(t)+5, min(t)+12])
        
        set_up_labels(i, ax[i], 
                      cases, seasons, num_cases, num_seasons, 
                      'time [days]', 'ACH [$h^{-1}$]')

    return plt

# Create the plots
fig = plot_normalized_flow_rates(r_all_cases_and_seasons)
save_plot(fig, "normalized_flow_all")

Total floor area 1662.66308594
Room height 2.74000000954
Volume      4555.69677734
Total floor area 1662.66308594
Room height 2.74000000954
Volume      4555.69677734
Total floor area 1662.66308594
Room height 2.74000000954
Volume      4555.69677734
Total floor area 1662.66308594
Room height 2.74000000954
Volume      4555.69677734
Total floor area 1662.66308594
Room height 2.74000000954
Volume      4555.69677734
Total floor area 1662.66308594
Room height 2.74000000954
Volume      4555.69677734


## Read annual results

This section also clears the old results to free up memory.

In [ ]:
# Free up storage, then read new data
r_base = list()
r_g36 = list()
r_g36_annual=get_results('annual_g36')
print("Read annual G36")
r_base_annual=get_results('annual_base')
print("Read annual base")

### Energy use

We convert the energy use to site electricity use, using an average COP for the chiller of 3.2, and a COP for heating of 4.0. These numbers are from https://portfoliomanager.energystar.gov/pdf/reference/Source%20Energy.pdf


In [ ]:
COPc=3.2
COPh=4.0

In [ ]:
def plot_energy(r_base, r_g36):
    from buildingspy.io.outputfile import Reader
    import matplotlib.pyplot as plt
    
    plt.clf()    

    # Conversion to kWh/m2
    conv = 1/3600./1000.
    readers = [r_base, r_g36]
    width = 0.5       # the width of the bars: can also be len(x) sequence

    hea    = [0., 0.]
    cooSen = [0., 0.]
    cooLat = [0., 0.]
    fan    = [0., 0.]
    cooLatBas = [0., 0.]
    coo = [0, 0]
    fanBas = [0., 0.]
    
    idx = [0, 1]
    for i in idx:
        hea[i]    =  readers[i].values('res.EHea')[1][-1] * conv / COPh
        cooSen[i] = -readers[i].values('res.ECooSen')[1][-1] * conv / COPc
        cooLat[i] = -readers[i].values('res.ECooLat')[1][-1] * conv / COPc
        coo[i] = cooSen[i] + cooLat[i]
        fan[i]    =  readers[i].values('res.EFan')[1][-1] * conv
        cooLatBas[i] = hea[i] + cooSen[i]
        fanBas[i]    = cooLatBas[i] + cooLat[i]
        
    p1 = plt.bar(idx, hea, width, color='r')
    p2 = plt.bar(idx, cooSen, width, bottom=hea, color='g')
    p3 = plt.bar(idx, cooLat, width, bottom=cooLatBas, color='b')
    p4 = plt.bar(idx, fan,    width, bottom=fanBas, color='k')
        
    plt.ylabel('site electricity use [kWh/(m2  a)]')
    plt.xticks([0, 1], ('base case', 'guideline 36'))
    plt.tick_params(axis=u'x', which=u'both',length=0)

    #plt.yticks(np.arange(0, 81, 10))
    plt.legend((p1[0], p2[0], p3[0], p4[0]), \
               ('heating', 'sensible cooling', 'latent cooling', 'fan'), \
              loc='upper right')
    
    
    save_plot(plt, "energy")
    
    # Write result to console and file
    eSit = [0, 0]
    for i in [0, 1]:
        eSit[i] = hea[i]+cooSen[i]+cooLat[i]+fan[i]

    str = """\
.. table:: Heating, cooling, fan and total site energy, and savings of guideline 36 case versus base case.

   ===================================== ===================================== ====================================== ====================================== =====
   :math:`E_{{h}} \quad [kWh/(m^2\,a)]`    :math:`E_{{c}} \quad [kWh/(m^2\,a)]`    :math:`E_{{f}} \quad [kWh/(m^2\,a)]`     :math:`E_{{tot}} \quad [kWh/(m^2\,a)]`     [%]
   ===================================== ===================================== ====================================== ====================================== =====
   {:37.4} {:37.4}  {:37.4}  {:37.4}   
   {:37.4} {:37.4}  {:37.4}  {:37.4}  {:4.3} 
   ===================================== ===================================== ====================================== ====================================== =====
  
  """.format(\
            hea[0], coo[0], fan[0], eSit[0], \
            hea[1], coo[1], fan[1], eSit[1], (1-eSit[1]/eSit[0])*100.)
    def save_rst(str, file_name):
        ''' Save the string `str` to the rst file `file_name.rst`
        '''
        print(str)
        with open(os.path.join("img", "{}.rst".format(file_name)), "w") as fil:
            fil.write(str)
        
    save_rst(str, "site_energy")
plot_energy(r_base_annual, r_g36_annual)

### Diagnostic output

In [ ]:
print("Average outdoor air intake, base {} kg/s".format(r_base_annual.mean('eco.port_Out.m_flow')))
print("Average outdoor air intake, G36  {} kg/s".format(r_g36_annual.mean( 'eco.port_Out.m_flow')))

print("CPUtime, base {} h".format(r_base_annual.values('CPUtime')[1][-1]/3600.))
print("CPUtime, G36  {} h".format(r_g36_annual.values( 'CPUtime')[1][-1]/3600.))

In [ ]:
(t, y) = r_base_annual.values('flo.sou.air.vol.T')
plt.clf()
plt=Plotter.boxplot(t=t, y=y-273.15, increment=3600, nIncrement=24)

# Decorate, save and show the plot
plt.xlabel('Time [h]') 
plt.ylabel(u'Room temperature [$^\circ$C]') 
plt.grid()
plt.savefig("roomTemperatures.png")
plt.savefig("roomTemperatures.pdf")
plt.title('South zone, base case')
#plt.show()

## Energy use

In [ ]:
2.8*60